In [33]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.autograd import Variable

In [ ]:
ColDir ="/home/data/collection/"

Define the network as class (from nn.Module) :

In [ ]:
class maxnet(nn.Module):
    def __init__(self):
        super(maxnet, self).__init__()
        self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1)),
                nn.Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1)),
                nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1000),
            nn.ReLU(True),
            nn.Linear(1000, 500),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [62]:
mymodel = maxnet()
input = Variable(torch.randn(1, 3, 225, 225))
print(mymodel.forward(input).size())

torch.Size([1, 500])


In [20]:
help(nn.MaxPool2d)

Help on class MaxPool2d in module torch.nn.modules.pooling:

class MaxPool2d(torch.nn.modules.module.Module)
 |  Applies a 2D max pooling over an input signal composed of several input
 |  planes.
 |  
 |  In the simplest case, the output value of the layer with input size :math:`(N, C, H, W)`,
 |  output :math:`(N, C, H_{out}, W_{out})` and :attr:`kernel_size` :math:`(kH, kW)`
 |  can be precisely described as:
 |  
 |  .. math::
 |  
 |      \begin{array}{ll}
 |      out(N_i, C_j, h, w)  = \max_{{m}=0}^{kH-1} \max_{{n}=0}^{kW-1}
 |                             input(N_i, C_j, stride[0] * h + m, stride[1] * w + n)
 |      \end{array}
 |  
 |  | If :attr:`padding` is non-zero, then the input is implicitly zero-padded on both sides
 |    for :attr:`padding` number of points
 |  | :attr:`dilation` controls the spacing between the kernel points. It is harder to describe,
 |    but this `link`_ has a nice visualization of what :attr:`dilation` does.
 |  
 |  The parameters :attr:`kernel_siz